# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint
# Рекомендуется использовать декоратор functools.wraps во всех декораторах,
# которые вы пишете сами.
from functools import wraps

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start}\n')
        return result
    return wrapper



## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'Функция вызвана с параметрами:\n{args}, {kwargs}\n')
        return result

    return wrapper



## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.cnt += 1
        result = func(*args, **kwargs)
        print(f'Функция была вызвана: {wrapper.cnt} раз\n')
        return result

    wrapper.cnt = 0
    return wrapper



## Задание 4

In [6]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func,
    чьи аргументы *args должны быть хешируемыми.
    """
    cache = {}

    @wraps(func)
    def fmemo(*args):
        if args in cache:
            # Если результат уже в кэше, возвращаем его
            return cache[args]
        else:
            # Если результата нет в кэше, вызываем функцию и запоминаем результат
            result = func(*args)
            cache[args] = result
            return result

    fmemo.cache = cache  # Добавляем доступ к кэшу
    return fmemo

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз\n"

print(word_count('whole'))

Время выполнения функции word_count: 0.34262225499969645

Функция вызвана с параметрами:
('whole',), {}

Функция была вызвана: 1 раз

Cлово whole встречается 176 раз



In [8]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [9]:
# измеряем время выполнения
%%time
fib(24)

CPU times: user 33.6 ms, sys: 0 ns, total: 33.6 ms
Wall time: 42 ms


46368

In [10]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [11]:
# измеряем время выполнения
%%time
fib(24)

CPU times: user 45 µs, sys: 0 ns, total: 45 µs
Wall time: 48.9 µs


46368